In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.callbacks import Callback
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.optimizers import SGD, RMSprop
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import random
import h5py

## Images loading

In [ ]:
directory_1 = '../data/train/1/'
directory_0 = '../data/train/0/'

img_x = 36
img_y = 36

batch_size = 10
epochs = 10

In [ ]:
X_1 = [cv2.imread(directory_1 + file, 0) for file in listdir(directory_1)]
y_1 = [1 for i in range(0, len(X_1))]

X_0 = [cv2.imread(directory_0 + file, 0) for file in listdir(directory_0)]
y_0 = [0 for i in range(0, len(X_0))]

data = np.array(X_0 + X_1)
target = np.array(y_0 + y_1)

data = data.reshape(data.shape[0], img_x, img_y, 1)

data = data.astype('float32')
data /= 255

target = to_categorical(target, 2)

In [ ]:
data.shape

In [ ]:
target[:5]

In [ ]:
print('Label = ', target[0])
plt.imshow(data[0].reshape(img_x, img_y), cmap = 'gray', interpolation = 'bicubic')
plt.show()

In [ ]:
lastindex = len(data) - 1
print('Label = ', target[lastindex])
plt.imshow(data[lastindex].reshape(img_x, img_y), cmap = 'gray', interpolation = 'bicubic')
plt.show()

## Model architecture 

In [ ]:
input_shape = (img_x, img_y, 1)

model = Sequential()

model.add(Conv2D(4, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(14, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(14, activation='relu'))
model.add(Dense(2, activation='softmax'))


In [ ]:
model.compile(loss=binary_crossentropy,
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])

## Model training

In [ ]:
# Splitting the dataset to get the training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.20, random_state=0)

# Defining a callback to get some logs at the end of each epoch
class AccuracyHistory(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

# Training the model
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,
          validation_split=0.20,
          callbacks=[history])

## Model testing

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print(model.metrics_names)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

plt.plot(range(1, 11), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

## Prediction

In [ ]:
predictions = model.predict(np.array(X_test[:10]), verbose=0)

for i in range(0, 10) :
    print('The result is : ', predictions[i], ' y = ', y_test[i])

    plt.imshow(X_test[i].reshape(36, 36), cmap = 'gray', interpolation = 'bicubic')
    plt.show()

In [ ]:
# Save the model
model.save('model_n.h5')